In [1]:
import scanpy as sc
import scvi
import numpy as np
import anndata as ad
from anndata import AnnData
from scipy import sparse
import pandas as pd

Global seed set to 0


In [2]:
#multiome = scvi.data.read_h5ad("multiome_gastr.h5ad")

In [3]:
rna_adata = scvi.data.read_h5ad("/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/jupyter_notebooks/anndata_rna.h5ad")
atac_adata = scvi.data.read_h5ad("/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/jupyter_notebooks/anndata_atac_peak_matrix.h5ad")

In [4]:
#adata = scvi.data.read_10x_multiome("filtered_feature_bc_matrix")

In [5]:
#adata.var_names_make_unique()
rna_adata.var_names_make_unique()
atac_adata.var_names_make_unique()

In [6]:
old = scvi.data.read_h5ad("/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/old_data_three_timepoints/anndata_rna.h5ad")

In [7]:
rna_adata = rna_adata[:,rna_adata.var.index.isin(old.var.index)]

In [8]:
annotations = pd.merge(old.var, rna_adata.var, on="gene", how = "inner")
annotations.index = old.var.index

In [9]:
annotations = annotations.drop(["gene", "Strand"], axis=1)

In [10]:
annotations = annotations.rename(columns={"Accession":"ID", "End":"end", "Start":"start", "Chromosome":"chr"})

In [11]:
rna_adata.var = annotations

In [12]:
matrix = sparse.hstack((rna_adata.X, atac_adata.X))

In [13]:
rna_df = rna_adata.var
#rna_df = rna_df.rename(columns = {"gene":"ID"})
rna_df["modality"] = ["Gene_expression" for i in range(len(rna_df))]
rna_df

,ID,chr,end,start,modality
Xkr4,ENSMUSG00000051951,1,3671498,3205901,Gene_expression
Gm1992,ENSMUSG00000089699,1,3513553,3466587,Gene_expression
Gm19938,ENSMUSG00000102331,1,3658904,3647309,Gene_expression
Gm37381,ENSMUSG00000102343,1,3986215,3905739,Gene_expression
Rp1,ENSMUSG00000025900,1,4409241,3999557,Gene_expression
...,...,...,...,...,...
AC124606.1,ENSMUSG00000095523,JH584299.1,840451,837364,Gene_expression
AC133095.2,ENSMUSG00000095475,JH584299.1,913083,910289,Gene_expression
AC133095.1,ENSMUSG00000094855,JH584299.1,924675,921942,Gene_expression
AC234645.1,ENSMUSG00000095019,JH584303.1,82689,81607,Gene_expression


In [14]:
atac_df = atac_adata.var
atac_df = atac_df.rename(columns={"idx":"ID"})
atac_df["modality"]=["Peaks" for i in range(len(atac_df))]

In [15]:
atac_df = atac_df.drop(["score"], axis = 1)

In [16]:
atac_df

,chr,start,end,ID,modality
chr1:3035600-3036200,chr1,3035600,3036200,chr1:3035600-3036200,Peaks
chr1:3062691-3063291,chr1,3062691,3063291,chr1:3062691-3063291,Peaks
chr1:3072272-3072872,chr1,3072272,3072872,chr1:3072272-3072872,Peaks
chr1:3191513-3192113,chr1,3191513,3192113,chr1:3191513-3192113,Peaks
chr1:3466250-3466850,chr1,3466250,3466850,chr1:3466250-3466850,Peaks
...,...,...,...,...,...
chrX:169883288-169883888,chrX,169883288,169883888,chrX:169883288-169883888,Peaks
chrX:169899093-169899693,chrX,169899093,169899693,chrX:169899093-169899693,Peaks
chrX:169906016-169906616,chrX,169906016,169906616,chrX:169906016-169906616,Peaks
chrX:169915632-169916232,chrX,169915632,169916232,chrX:169915632-169916232,Peaks


In [17]:
df = rna_df.append(atac_df)

In [18]:
multiome = AnnData(
X = matrix,
obs = atac_adata.obs,
var = df,
uns = rna_adata.uns)

In [19]:
multiome.shape

(45991, 212744)

In [20]:
adata_mvi = scvi.data.organize_multiome_anndatas(multiome)


In [21]:
adata_mvi = adata_mvi[:, adata_mvi.var["modality"].argsort()].copy()


In [22]:
print(adata_mvi.shape)
sc.pp.filter_genes(adata_mvi, min_cells=int(adata_mvi.shape[0] * 0.01))
print(adata_mvi.shape)

(45991, 212744)
(45991, 186627)


In [23]:
scvi.model.MULTIVI.setup_anndata(adata_mvi, batch_key='modality')

/omics/groups/OE0533/internal/katharina/scvi2/lib/python3.7/site-packages/scvi/data/_utils.py:102: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  "Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization."


In [26]:
mvi = scvi.model.MULTIVI(
    adata_mvi,
    n_genes=(adata_mvi.var['modality']=='Gene_expression').sum(),
    n_regions=(adata_mvi.var['modality']=='Peaks').sum(),
)
mvi.view_anndata_setup()

Anndata setup with scvi-tools version 0.15.0.

Setup via `MULTIVI.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': 'modality',
│   'labels_key': None,
│   'size_factor_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

         Summary Statistics          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     Summary Stat Key     ┃ Value  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│         n_cells          │ 45991  │
│          n_vars          │ 186627 │
│         n_batch          │   1    │
│         n_labels         │   1    │
│ n_extra_categorical_covs │   0    │
│ n_extra_continuous_covs  │   0    │
└──────────────────────────┴────────┘

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    labels    │ adata.obs['_scvi_labels'] │
└──────────────┴───────────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

In [ ]:
mvi.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Epoch 1/500:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
#multiome_mvi = scvi.data.organize_multiome_anndatas(multiome)

In [ ]:
#multiome_mvi.var

In [ ]:
#adata_mvi = scvi.data.organize_multiome_anndatas(multiome)

In [ ]:
#multiome.write("multiome_gastr.h5ad", compression="gzip")

In [ ]:
#multiome_mvi = multiome_mvi[:, multiome_mvi.var["modality"].argsort()].copy()

In [ ]:
#multiome_mvi.var

Filter features which appear in fewer than 1% of cells.

The main batch key should correspond to the modality of the cells. All other batch annotations are added via categorical_covariate_keys.

In [ ]:
test = scvi.data.organize_multiome_anndatas(multiome)

In [ ]:
scvi.model.MULTIVI.setup_anndata(adata)#,
                                # batch_key = "modality")
                                 #categorical_covariate_keys=["sample"])

We need to specify how many features are genes and how many are genomic regions. 

In [ ]:
mvi = scvi.model.MULTIVI(
    adata,
    n_genes=(adata.var['modality']=='Gene_expression').sum(),
    n_regions=(adata.var['modality']=='Peaks').sum()
    #dropout_rate=0.2,
    #n_layers_encoder=2,
    #n_layers_decoder=2,
    #n_latent=20
)

arches_params = dict(
    n_genes=(multiome.var["modality"]=="Gene_expression").sum(),
    n_regions=(multiome.var["modality"]=="Peaks").sum(),
    #dropout_rate=0.2,
    #n_layers_encoder=2,
    #n_layers_decoder=2,
    #n_latent=20,
    #latent_distribution='normal'
)

mvi = scvi.model.MULTIVI(
    multiome,
    **arches_params
)
mvi.view_anndata_setup()

In [ ]:
mvi.train()